# Convergence of Monte Carlo sampling for invariant measure

### 0. Load packages

In [12]:
using PyPlot, FileIO, JLD2, Printf
using StatsBase, LinearAlgebra, Statistics
using BernoulliMap, Distributions, OptimalTransport
using RandomNumbers

In [113]:
# bins
βs = [3/2,5/4,4/3,6/5]                                                
nβs = length(βs)
bin_width = 2.5e-2
bin_edges = collect(0:bin_width:1)
bin_mids = bin_edges[1:end-1] + diff(bin_edges)/2
nbins = length(bin_edges)-1

40

In [114]:
# Sample analytical invariant measure at bins
hxs = fill(0.0,nβs,nbins)
x = collect(0:1e-6:1)

for (iβ,β) in enumerate(βs)
    H = [sum(hofbauer.(x[(x .>= bin_edges[i]) .& (x .< bin_edges[i+1])],β)) for i in 1:nbins]
    hxs[iβ,:] = H / sum(H) / bin_width
end

## 2. Comparison to Monte Carlo-based invariant measures

In [76]:
# Calculate histograms
Nm = 5                                   # repeat times for uncertainty
Nens_size = round.(Int,1e1.^(0:0.2:4))   # start from Nens random initial conditions
Nsteps = 10000                           # integrate for Nsteps time steps
Nspinup = 5000                           # discard Nspinsup time steps

# each for different β
βs = [3/2,5/4,4/3,6/5]                                                
Nβs = length(βs)
Nenssizes = length(Nens_size)

# preallocate all histograms
Histos = fill(0.0,Nm,Nenssizes,Nβs,nbins)

for im in 1:Nm
    print("$im: ")
    for (inens,Nens) in enumerate(Nens_size)
        print("N=$inens,")
        for (iβ,β) in enumerate(βs)
            X = fill(0.0,Nens,Nsteps)
            for iens in 1:Nens
                f = randfloat(Float32)
                X[iens,:] = bernoulli_map(f,Float32(β),Nsteps+Nspinup)[Nspinup+2:end]
            end
            H = fit(Histogram,vec(X),bin_edges).weights
            Histos[im,inens,iβ,:] = H / sum(H) / bin_width
        end
    end
end

1: N=1,N=2,N=3,N=4,N=5,N=6,N=7,N=8,N=9,N=10,N=11,N=12,N=13,N=14,N=15,N=16,N=17,N=18,N=19,N=20,N=21,2: N=1,N=2,N=3,N=4,N=5,N=6,N=7,N=8,N=9,N=10,N=11,N=12,N=13,N=14,N=15,N=16,N=17,N=18,N=19,N=20,N=21,3: N=1,N=2,N=3,N=4,N=5,N=6,N=7,N=8,N=9,N=10,N=11,N=12,N=13,N=14,N=15,N=16,N=17,N=18,N=19,N=20,N=21,4: N=1,N=2,N=3,N=4,N=5,N=6,N=7,N=8,N=9,N=10,N=11,N=12,N=13,N=14,N=15,N=16,N=17,N=18,N=19,N=20,N=21,5: N=1,N=2,N=3,N=4,N=5,N=6,N=7,N=8,N=9,N=10,N=11,N=12,N=13,N=14,N=15,N=16,N=17,N=18,N=19,N=20,N=21,

# Wasserstein distances as a function of number of ensemble members

In [115]:
W = fill(0.0,Nm,Nenssizes,Nβs)

for (iβ,β) in enumerate(βs)
    
    # analytical invariant measure
    ν = DiscreteNonParametric(bin_mids,hxs[iβ,:]/nbins)
    
    for im in 1:Nm
        for (inens,Nens) in enumerate(Nens_size)
            μ = DiscreteNonParametric(bin_mids,Histos[im,inens,iβ,:]/nbins)

            W[im,inens,iβ] = wasserstein(ν,μ)
        end
    end
end

LoadError: ArgumentError: DiscreteNonParametric: the condition length(vs) == length(ps) is not satisfied.

In [78]:
Wmean = mean(W,dims=1)[1,:,:]
Wmin = minimum(W,dims=1)[1,:,:]
Wmax = maximum(W,dims=1)[1,:,:];

# Exact invariant measures from sampling from all initial conditions

In [102]:
# Load
orbits1 = load("../data/orbits_Float32_1.5f0.jld2","orbits")          # beta=3/2
orbits2 = load("../data/orbits_Float32_1.25f0.jld2","orbits")         # beta=5/4
orbits3 = load("../data/orbits_Float32_1.3333334f0.jld2","orbits")    # beta=4/3
orbits4 = load("../data/orbits_Float32_1.2f0.jld2","orbits")          # beta=6/5

9-element Vector{Orbit{Float32}}:
 Orbit{Float32,β=1.2f0}(length=     1, min=           0.0f0, basin=  5.513429641723633e-7)
 Orbit{Float32,β=1.2f0}(length=     1, min=         1.0f-45, basin= 1.401298464324817e-45)
 Orbit{Float32,β=1.2f0}(length=     1, min=         3.0f-45, basin= 1.401298464324817e-45)
 Orbit{Float32,β=1.2f0}(length=    16, min=   0.057180643f0, basin= 1.3462480772294033e-6)
 Orbit{Float32,β=1.2f0}(length=   135, min=   0.008523345f0, basin=    0.0141749985298142)
 Orbit{Float32,β=1.2f0}(length=   329, min=  0.0030605793f0, basin=  0.004048023040238647)
 Orbit{Float32,β=1.2f0}(length=   792, min=  0.0016326904f0, basin=  0.007872094328647666)
 Orbit{Float32,β=1.2f0}(length=  2867, min=  0.0010089874f0, basin=   0.13644943310914648)
 Orbit{Float32,β=1.2f0}(length= 10190, min=    5.1259995f-5, basin=    0.8374535534011116)

In [116]:
# CONVERT ORBITS TO HISTOGRAMS
Histow = fill(0.0,Nβs,nbins)

for (io,orbits) in enumerate([orbits1,orbits2,orbits3,orbits4])
    
    norbits = length(orbits)
    Histows = fill(0.0,norbits,nbins)
    
    for (ioo,orbit) in enumerate(orbits)
        X = bernoulli_map(orbit.min,orbit.β,orbit.length-1)
        H = fit(Histogram,X,bin_edges).weights
        Histows[ioo,:] = H / sum(H) / bin_width
    end

    # weighted means
    basins = [orbit.b for orbit in orbits]
    Histow[io,:] = mean(Histows,aweights(basins),dims=1)
end

In [117]:
# Wasserstein distances
Wana = fill(0.0,Nβs)

for (iβ,β) in enumerate(βs)
    
    # analytical invariant measure
    ν = DiscreteNonParametric(bin_mids,hxs[iβ,:]/nbins)
    μ = DiscreteNonParametric(bin_mids,Histow[iβ,:]/nbins)
    Wana[iβ] = wasserstein(ν,μ)
end

In [1]:
# PLOT
fig,axs = subplots(2,2,sharex=true,sharey=true,figsize=(8,4))

for (iax,ax) in enumerate(axs)
    ax.set_xscale("log")
    ax.plot(Nens_size,Wmean[:,iax],"teal",lw=2,label="Mean")
    ax.fill_between(Nens_size,Wmin[:,iax],Wmax[:,iax],color="teal",alpha=0.3,label="Min-max range")
    ax.axhline(Wana[iax],color="k",lw=1,ls="--",label="Exact")
end

for (iβ,β) in enumerate(["3/2","5/4","4/3","6/5"])
    axs[iβ].set_title("Monte Carlo convergence, β=$β",loc="left")
end

for (iabc,abc) in enumerate(["a","c","b","d"])
    axs[iabc].set_title(abc,loc="right",fontweight="bold")
end

axs[1,2].legend()

axs[1,1].set_ylabel("Wasserstein distance")
axs[2,1].set_ylabel("Wasserstein distance")
axs[2,1].set_xlabel("N")
axs[2,2].set_xlabel("N")
axs[1,1].set_xlim(1,maximum(Nens_size))
axs[1,1].set_ylim(-0.0005,0.02)

tight_layout()
# savefig("../plots/convergence.png")

LoadError: UndefVarError: subplots not defined